# Methodology

This section outlines our approach to evaluating the impact of different imputation methods on predictive modeling for heart failure patients in MIMIC-IV.

## 1. Data Preprocessing and Feature Selection

### Feature Selection Pipeline
- Initial feature selection using LASSO regression to identify the most important predictors
- Further refinement using XGBoost feature importance
- Final feature set used across all imputation methods for fair comparison

In [8]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV
import xgboost as xgb
from sklearn.model_selection import train_test_split

def feature_selection_pipeline(data, target_col):
    # Split data
    X = data.drop(columns=[target_col])
    y = data[target_col]
    
    # LASSO feature selection
    lasso = LassoCV(cv=5)
    lasso.fit(X, y)
    
    # Get non-zero coefficients
    lasso_features = X.columns[lasso.coef_ != 0]
    
    # XGBoost feature importance
    xgb_model = xgb.XGBClassifier()
    xgb_model.fit(X[lasso_features], y)
    
    # Get feature importance
    importance = pd.DataFrame({
        'feature': lasso_features,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    return importance

## 2. Missing Data Handling

### Imputation Methods
We evaluate three different imputation approaches:
1. **Mean/Mode Imputation**: Simple baseline method
2. **Regression-based Imputation**: Using predictive models for each feature
3. **GPLVM Imputation**: Advanced deep learning-based approach

### Missingness Scenarios
- Full data (0% missing)
- Subset with 0% missing values
- Subset with 20% missing values
- Subset with 40% missing values

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

def create_missing_data(data, missing_percentage):
    # Create missing values in the dataset
    mask = np.random.random(data.shape) < missing_percentage
    data_missing = data.copy()
    data_missing[mask] = np.nan
    return data_missing

def mean_mode_imputation(data):
    # Separate numeric and categorical columns
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    categorical_cols = data.select_dtypes(include=['object']).columns
    
    # Create imputers
    numeric_imputer = SimpleImputer(strategy='mean')
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    
    # Impute data
    data_imputed = data.copy()
    if len(numeric_cols) > 0:
        data_imputed[numeric_cols] = numeric_imputer.fit_transform(data[numeric_cols])
    if len(categorical_cols) > 0:
        data_imputed[categorical_cols] = categorical_imputer.fit_transform(data[categorical_cols])
    
    return data_imputed

def regression_imputation(data):
    # Use Random Forest for imputation
    imputer = IterativeImputer(
        estimator=RandomForestRegressor(),
        max_iter=10,
        random_state=42
    )
    
    # Impute only numeric columns
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    data_imputed = data.copy()
    data_imputed[numeric_cols] = imputer.fit_transform(data[numeric_cols])
    
    return data_imputed

## GPLVM Implementation Details

The Gaussian Process Latent Variable Model (GPLVM) implementation includes:

1. **Model Structure**:
   - Sparse GP regression for efficient computation
   - RBF kernel for smooth latent space representation
   - Two-dimensional latent space for visualization

2. **Key Components**:
   - Prior mean initialization for latent variables
   - Inducing points for sparse approximation
   - Automatic guide for variational inference

3. **Features**:
   - Imputation of missing values with uncertainty estimates
   - Visualization of training progress and latent space
   - Support for both continuous and categorical variables

4. **Usage Example**:
```python
# Example usage
imputed_data, uncertainty = gplvm_imputation(
    data=your_data,
    latent_dim=2,
    num_inducing=32,
    num_steps=4000
)


1. Provides a complete GPLVM-based imputation method
2. Includes uncertainty estimation for imputed values
3. We use package from https://github.com/pyro-ppl/pyro/blob/dev/tutorial/source/gplvm.ipynb

In [5]:
import torch
import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist
import pyro.ops.stats as stats
from torch.nn import Parameter
import matplotlib.pyplot as plt

def gplvm_imputation(data, latent_dim=2, num_inducing=32, num_steps=4000):
    """
    GPLVM-based imputation for missing values
    
    Parameters:
    -----------
    data : pandas DataFrame
        Input data with missing values
    latent_dim : int
        Dimension of the latent space
    num_inducing : int
        Number of inducing points for sparse GP
    num_steps : int
        Number of optimization steps
    
    Returns:
    --------
    imputed_data : pandas DataFrame
        Data with imputed values
    uncertainty : pandas DataFrame
        Uncertainty estimates for imputed values
    """
    # Convert data to torch tensor
    data_tensor = torch.tensor(data.values, dtype=torch.get_default_dtype())
    y = data_tensor.t()
    
    # Create prior mean for latent variables
    X_prior_mean = torch.zeros(y.size(1), latent_dim)
    
    # Define RBF kernel
    kernel = gp.kernels.RBF(input_dim=latent_dim, lengthscale=torch.ones(latent_dim))
    
    # Initialize latent variables
    X = Parameter(X_prior_mean.clone())
    
    # Initialize inducing points
    Xu = stats.resample(X_prior_mean.clone(), num_inducing)
    
    # Create sparse GP model
    gplvm = gp.models.SparseGPRegression(
        X, y, kernel, Xu,
        noise=torch.tensor(0.01),
        jitter=1e-5
    )
    
    # Set up prior for X
    gplvm.X = pyro.nn.PyroSample(
        dist.Normal(X_prior_mean, 0.1).to_event()
    )
    
    # Set up guide
    gplvm.autoguide("X", dist.Normal)
    
    # Train the model
    losses = gp.util.train(gplvm, num_steps=num_steps)
    
    # Get imputed values and uncertainty
    gplvm.mode = "guide"
    X = gplvm.X_loc.detach().numpy()
    
    # Reconstruct data with uncertainty
    imputed_data = data.copy()
    uncertainty = pd.DataFrame(index=data.index, columns=data.columns, dtype=float)
    
    # Use the learned latent space to impute missing values
    for i in range(data.shape[1]):
        if data.iloc[:, i].isnull().any():
            # Get predictions for missing values
            pred_mean, pred_var = gplvm.forward(X)
            imputed_data.iloc[:, i] = pred_mean.detach().numpy()
            uncertainty.iloc[:, i] = pred_var.detach().numpy()
    
    return imputed_data, uncertainty

def plot_gplvm_results(losses, X, labels=None):
    """
    Plot GPLVM training results and latent space visualization
    """
    # Plot loss curve
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(losses)
    plt.title("GPLVM Training Loss")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    
    # Plot latent space
    plt.subplot(1, 2, 2)
    if labels is not None:
        colors = plt.get_cmap("tab10").colors
        for i, label in enumerate(labels.unique()):
            mask = labels == label
            plt.scatter(X[mask, 0], X[mask, 1], c=[colors[i]], label=label)
        plt.legend()
    else:
        plt.scatter(X[:, 0], X[:, 1])
    
    plt.title("GPLVM Latent Space")
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.tight_layout()
    plt.show()

The Constrained SVM approach is a method that takes into account the uncertainty in the imputed values when making predictions. 

Instead of using a single imputed dataset, we generate multiple possible datasets based on the uncertainty estimates from GPLVM

Each dataset represents a different possible realization of the true values
We train an SVM on each dataset and then select the optimal model

In [7]:
def constrained_svm(X, y, uncertainty, n_samples=10):
    """
    Constrained SVM implementation that incorporates uncertainty
    
    Parameters:
    -----------
    X : numpy array
        Feature matrix (including imputed values)
    y : numpy array
        Target labels
    uncertainty : numpy array
        Uncertainty estimates from GPLVM
    n_samples : int
        Number of possible datasets to generate
    """
    # Store all trained models and their performance
    models = []
    scores = []
    
    for _ in range(n_samples):
        # Generate a new dataset by sampling from uncertainty distribution
        X_sampled = X + np.random.normal(0, np.sqrt(uncertainty))
        
        # Train SVM on this sampled dataset
        model = SVC(kernel='rbf')
        model.fit(X_sampled, y)
        
        # Evaluate model performance
        score = cross_val_score(model, X_sampled, y, cv=5).mean()
        
        models.append(model)
        scores.append(score)
    
    # Select the best performing model
    best_idx = np.argmax(scores)
    best_model = models[best_idx]
    
    return best_model, scores[best_idx]

In [8]:
def visualize_constrained_svm(X, y, uncertainty, best_model):
    """
    Visualize the constrained SVM process
    """
    plt.figure(figsize=(15, 5))
    
    # Plot 1: Original data with uncertainty
    plt.subplot(131)
    plt.scatter(X[:, 0], X[:, 1], c=y, alpha=0.5)
    plt.title("Original Data with Uncertainty")
    
    # Plot 2: Sampled datasets
    plt.subplot(132)
    for _ in range(5):  # Show 5 sampled datasets
        X_sampled = X + np.random.normal(0, np.sqrt(uncertainty))
        plt.scatter(X_sampled[:, 0], X_sampled[:, 1], c=y, alpha=0.2)
    plt.title("Multiple Sampled Datasets")
    
    # Plot 3: Decision boundary of best model
    plt.subplot(133)
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                         np.arange(y_min, y_max, 0.1))
    Z = best_model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.4)
    plt.scatter(X[:, 0], X[:, 1], c=y, alpha=0.8)
    plt.title("Best Model Decision Boundary")
    
    plt.tight_layout()
    plt.show()

In [9]:
def complete_constrained_svm_analysis(data, target_col, missing_percentage=0.2):
    """
    Complete analysis pipeline combining GPLVM and cSVM
    """
    # 1. Create missing data
    data_missing = create_missing_data(data, missing_percentage)
    
    # 2. GPLVM imputation with uncertainty
    imputed_data, uncertainty = gplvm_imputation(data_missing)
    
    # 3. Prepare data for SVM
    X = imputed_data.drop(columns=[target_col])
    y = imputed_data[target_col]
    
    # 4. Train constrained SVM
    best_model, best_score = constrained_svm(X, y, uncertainty)
    
    # 5. Visualize results
    visualize_constrained_svm(X, y, uncertainty, best_model)
    
    return best_model, best_score, imputed_data, uncertainty

By considering multiple possible imputed datasets, the model becomes more robust to imputation uncertainty

Uncertainty Integration: The uncertainty estimates from GPLVM are directly used in the model selection process

Model Selection: The best model is selected based on its performance across different possible realizations of the data

## 3. Model Development and Evaluation

### Model Pipeline
- Baseline Models:
  - Logistic Regression
  - Random Forest
- Primary Model: XGBoost

### Evaluation Metrics
- Classification Metrics:
  - AUC-ROC
  - F1 Score
  - Precision
  - Recall
- Model Stability:
  - Feature importance consistency across missingness levels

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

def train_evaluate_models(X_train, X_test, y_train, y_test):
    # Define models
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000),
        'Random Forest': RandomForestClassifier(),
        'XGBoost': xgb.XGBClassifier()
    }
    
    # Define parameter grids for GridSearchCV
    param_grids = {
        'Logistic Regression': {'C': [0.1, 1, 10]},
        'Random Forest': {'n_estimators': [100, 200], 'max_depth': [None, 10]},
        'XGBoost': {'n_estimators': [100, 200], 'max_depth': [3, 6]}
    }
    
    results = {}
    
    for name, model in models.items():
        # Perform grid search
        grid_search = GridSearchCV(model, param_grids[name], cv=5, scoring='roc_auc')
        grid_search.fit(X_train, y_train)
        
        # Get best model
        best_model = grid_search.best_estimator_
        
        # Make predictions
        y_pred = best_model.predict(X_test)
        y_pred_proba = best_model.predict_proba(X_test)[:, 1]
        
        # Calculate metrics
        results[name] = {
            'auc_roc': roc_auc_score(y_test, y_pred_proba),
            'f1': f1_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred)
        }
    
    return results

## 4. Model Interpretation

### SHAP Analysis
- Compare feature importance across different missingness levels
- Identify stable vs. unstable features
- Assess the impact of imputation on feature importance

In [1]:
import shap
import numpy as np
import warnings

def analyze_feature_importance(model, X, feature_names):
    # Suppress warnings about NumPy version
    warnings.filterwarnings('ignore', category=UserWarning)
    
    try:
        # Create SHAP explainer
        explainer = shap.TreeExplainer(model)
        
        # Calculate SHAP values
        shap_values = explainer.shap_values(X)
        
        # Create summary plot
        shap.summary_plot(shap_values, X, feature_names=feature_names)
        
        # Calculate feature importance
        importance = pd.DataFrame({
            'feature': feature_names,
            'importance': np.abs(shap_values).mean(0)
        }).sort_values('importance', ascending=False)
        
    except Exception as e:
        print(f"Warning: SHAP analysis encountered an error: {str(e)}")
        print("Using alternative feature importance method...")
        
        # Fallback to model's built-in feature importance
        if hasattr(model, 'feature_importances_'):
            importance = pd.DataFrame({
                'feature': feature_names,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
        else:
            # If no feature importance is available, use coefficients
            if hasattr(model, 'coef_'):
                importance = pd.DataFrame({
                    'feature': feature_names,
                    'importance': np.abs(model.coef_[0])
                }).sort_values('importance', ascending=False)
            else:
                print("No feature importance method available for this model")
                return None
    
    return importance